# Text Model Template

In [10]:
from langchain import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt_template.format(adjective="gruesome", content="goats")

'Tell me a gruesome joke about goats.'

# Chat Template

In [11]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

messages = template.format_messages(
    name="Bob",
    user_input="What is your name?"
)

ValidationError: 8 validation errors for ChatPromptTemplate
messages -> 0
  value is not a valid dict (type=type_error.dict)
messages -> 0
  value is not a valid dict (type=type_error.dict)
messages -> 1
  value is not a valid dict (type=type_error.dict)
messages -> 1
  value is not a valid dict (type=type_error.dict)
messages -> 2
  value is not a valid dict (type=type_error.dict)
messages -> 2
  value is not a valid dict (type=type_error.dict)
messages -> 3
  value is not a valid dict (type=type_error.dict)
messages -> 3
  value is not a valid dict (type=type_error.dict)

In [12]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessage, HumanMessagePromptTemplate

template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that re-writes the user's text to "
                "sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()
llm(template.format_messages(text='i dont like eating tasty things.'))

AIMessage(content='I absolutely adore indulging in delicious treats!', additional_kwargs={}, example=False)

Creating a chain

In [13]:
import os
from getpass import getpass
from langchain import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {topic}"
)

prompt_template.format(adjective="offensive", topic="racism")

'Tell me a offensive joke about racism'

Imports

In [14]:
from google.cloud import aiplatform
from langchain.chat_models import ChatVertexAI
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.schema import HumanMessage, SystemMessage
from pydantic import BaseModel
from typing import List
import vertexai

In [15]:
PROJECT_ID = "gen-ai-sandbox"  # @param {type:"string"}
vertexai.init(project=PROJECT_ID, location="us-central1")

In [16]:
# Utility functions for Embeddings API with rate limiting
def rate_limit(max_per_minute):
    period = 60 / max_per_minute
    print("Waiting")
    while True:
        before = time.time()
        yield
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elapsed)
        if sleep_time > 0:
            print(".", end="")
            time.sleep(sleep_time)


class CustomVertexAIEmbeddings(VertexAIEmbeddings, BaseModel):
    requests_per_minute: int
    num_instances_per_batch: int

    # Overriding embed_documents method
    def embed_documents(self, texts: List[str]):
        limiter = rate_limit(self.requests_per_minute)
        results = []
        docs = list(texts)

        while docs:
            # Working in batches because the API accepts maximum 5
            # documents per request to get embeddings
            head, docs = (
                docs[: self.num_instances_per_batch],
                docs[self.num_instances_per_batch :],
            )
            chunk = self.client.get_embeddings(head)
            results.extend(chunk)
            next(limiter)

        return [r.values for r in results]

Defining all the models here

In [32]:
# LLM model
llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

# Chat
chat = ChatVertexAI()

# Embedding
# EMBEDDING_QPM = 100
# EMBEDDING_NUM_BATCH = 5
# embeddings = CustomVertexAIEmbeddings(
#     requests_per_minute=EMBEDDING_QPM,
#     num_instances_per_batch=EMBEDDING_NUM_BATCH,
# )

In [33]:
my_text = "What day comes after Friday?"

print(llm(my_text))

AttributeError: 'google.protobuf.pyext._message.RepeatedCompositeCo' object has no attribute 'WhichOneof'

In [28]:
# palm api stuff
from langchain.llms.vertexai import VertexAI
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

# llm = VertexAI(model_name="code-bison")
# llm("say hello")
# llm = VertexAI(model_name="text-bison")
llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

In [29]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("colorful socks"))



Bright Toe Socks
